In [2]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
ds=pd.read_csv("C:/Users/Asus/Downloads/cyberai/train.csv")
ds.head(100)

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...
...,...,...,...
95,Online Financial Fraud,Fraud CallVishing,\r\n n...
96,Online Financial Fraud,Internet Banking Related Fraud,
97,Online Financial Fraud,UPI Related Frauds,FRAUD THROUGH UPI FRAUDPHONE PE\r\nBANK NAME ...
98,RapeGang Rape RGRSexually Abusive Content,NaN,Respected Sir\r\n\r\nA very serious matter I w...


In [11]:
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [14]:
# Load train and test datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Combine text features
train_data["text"] = train_data["sub_category"] + " " + train_data["crimeaditionalinfo"]
test_data["text"] = test_data["sub_category"] + " " + test_data["crimeaditionalinfo"]

# Encode labels
train_data["label"] = train_data["category"].astype("category").cat.codes
test_data["label"] = test_data["category"].astype("category").cat.codes


In [18]:
print(train_data["text"].isnull().sum())  # Check for NaN values
print(test_data["text"].isnull().sum())  # Check for NaN values

6612
2242


In [19]:
train_data["text"] = train_data["text"].fillna("").astype(str)
test_data["text"] = test_data["text"].fillna("").astype(str)


In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

C:\Users\Asus\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
train_encodings = tokenizer(list(train_data["text"]), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_data["text"]), truncation=True, padding=True, max_length=128)

In [22]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(train_encodings, list(train_data["label"]))
test_dataset = TextDataset(test_encodings, list(test_data["label"]))


In [23]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model with classification head
num_classes = len(train_data["category"].unique())
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Asus\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: jainrivamait (jainrivamait-brightchamps). Use `wandb login --relogin` to force relogin


C:\Users\Asus\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,0.020900,11.322732
2,0.083600,11.408836
3,0.104800,12.338241


TrainOutput(global_step=17568, training_loss=0.10774210107619644, metrics={'train_runtime': 25846.0529, 'train_samples_per_second': 10.874, 'train_steps_per_second': 0.68, 'total_flos': 1.848952462722509e+16, 'train_loss': 0.10774210107619644, 'epoch': 3.0})

In [31]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

# Get predictions
preds_output = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

# Ground truth labels
true_labels = list(test_data["label"])

# Accuracy
accuracy = accuracy_score(true_labels, preds)

# Precision and Recall (macro-averaged for multi-class tasks)
precision = precision_score(true_labels, preds, average="macro")
recall = recall_score(true_labels, preds, average="macro")

C:\Users\Asus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
f1_score=(2*precision*recall)/(precision+recall)

In [34]:
# Print metrics

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"F1-Score:{f1_score:.4f}")

Accuracy: 1.0
Precision (Macro): 0.67
Recall (Macro): 0.71
F1-Score:0.69


In [35]:
def predict_text(input_text, model, tokenizer, label_map):
    """
    Predict the category of the given text.
    
    Args:
        input_text (str): The input text to classify.
        model: The fine-tuned BERT model.
        tokenizer: The tokenizer used for BERT.
        label_map (dict): A dictionary mapping label indices to category names.
    
    Returns:
        str: Predicted category.
    """
    # Tokenize input text
    encodings = tokenizer(input_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**encodings)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()
    
    # Return the corresponding category
    return label_map[predicted_label]


In [36]:
# Map label indices to category names
label_map = {idx: label for idx, label in enumerate(train_data["category"].astype("category").cat.categories)}
